# Profiling to find Bottlenecks
## iPython Profile

In [15]:
# save this function in file foo.py
def foo(n):
    phrase = 'repeat me'
    pmul = phrase * n
    pjoi = ''.join([phrase for x in xrange(n)])
    pinc = ''
    for x in xrange(n):
        pinc += phrase
    del pmul, pjoi, pinc
    
from foo import foo
%reload_ext line_profiler

%load_ext memory_profiler

### Time Profile

In [16]:
%time foo(100000)

CPU times: user 26.3 ms, sys: 4.33 ms, total: 30.6 ms
Wall time: 30.8 ms


In [17]:
%timeit foo(100000)

10 loops, best of 3: 24.7 ms per loop


In [19]:
%prun foo(100000)

 4 function calls in 0.063 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.061    0.061    0.063    0.063 foo.py:1(foo)
        1    0.002    0.002    0.002    0.002 {method 'join' of 'str' objects}
        1    0.000    0.000    0.063    0.063 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [11]:
%lprun -f foo foo(100000)

Timer unit: 1e-06 s

Total time: 0.170241 s
File: foo.py
Function: foo at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def foo(n):
     2         1           14     14.0      0.0      phrase = 'repeat me'
     3         1         1077   1077.0      0.6      pmul = phrase * n
     4    100001        67130      0.7     39.4      pjoi = ''.join([phrase for x in xrange(n)])
     5         1            3      3.0      0.0      pinc = ''
     6    100001        41873      0.4     24.6      for x in xrange(n):
     7    100000        60018      0.6     35.3          pinc += phrase
     8         1          126    126.0      0.1      del pmul, pjoi, pinc

### Memory Profile

In [ ]:
%mprun -f foo foo(100000)

Filename: foo.py

Line #    Mem usage    Increment   Line Contents
================================================
     1     65.4 MiB      0.0 MiB   def foo(n):
     2     65.4 MiB      0.0 MiB       phrase = 'repeat me'
     3     65.4 MiB      0.0 MiB       pmul = phrase * n
     4     65.4 MiB      0.0 MiB       pjoi = ''.join([phrase for x in xrange(n)])
     5     65.2 MiB     -0.2 MiB       pinc = ''
     6     70.1 MiB      4.9 MiB       for x in xrange(n):
     7     70.1 MiB      0.0 MiB           pinc += phrase
     8     67.6 MiB     -2.5 MiB       del pmul, pjoi, pinc

In [14]:
%memit foo(100000)

peak memory: 68.15 MiB, increment: 0.55 MiB


## Other methods

In [5]:
"""Julia set generator with timing decorator"""
import time
from functools import wraps

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193


def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output


def calc_pure_python(draw_output, desired_width, max_iterations):
    """Create a list of complex co-ordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (float(x2 - x1) / float(desired_width))
    y_step = (float(y1 - y2) / float(desired_width))
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of co-ordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print "Length of x:", len(x)
    print "Total elements:", len(zs)
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print calculate_z_serial_purepython.func_name + " took", secs, "seconds"

    # this sum is expected for 1000^2 grid with 300 iterations
    assert sum(output) == 33219980


# Calculate the Julia set using a pure Python solution with
# reasonable defaults for a laptop
# set draw_output to True to use PIL to draw an image
calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 14.7274129391 seconds


## Using cProfiles

Copy above cell to a file name test_cpu_time.py
run it in terminal by this command: *python -m cProfile -o profile.stats test_cpu_time.py*

In [6]:
import pstats
p = pstats.Stats("profile.stats")
p.print_stats()

Wed Mar 22 16:04:50 2017    profile.stats

         36221992 function calls in 22.007 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  2002000    0.192    0.000    0.192    0.000 {method 'append' of 'list' objects}
        2    0.000    0.000    0.000    0.000 {time.time}
        1    0.059    0.059   22.007   22.007 test_cpu_time.py:1(<module>)
        4    0.000    0.000    0.000    0.000 {len}
        1    1.246    1.246   21.948   21.948 test_cpu_time.py:24(calc_pure_python)
 34219980    3.692    0.000    3.692    0.000 {abs}
        1    0.007    0.007    0.007    0.007 {sum}
        1   16.739   16.739   20.503   20.503 test_cpu_time.py:10(calculate_z_serial_purepython)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.073    0.073    0.073    0.073 {range}




<pstats.Stats instance at 0x115d2eb00>

In [7]:
p.print_callers()

   Random listing order was used

Function                                            was called by...
                                                        ncalls  tottime  cumtime
{method 'append' of 'list' objects}                 <- 2002000    0.192    0.192  test_cpu_time.py:24(calc_pure_python)
{time.time}                                         <-       2    0.000    0.000  test_cpu_time.py:24(calc_pure_python)
test_cpu_time.py:1(<module>)                        <-
{len}                                               <-       2    0.000    0.000  test_cpu_time.py:10(calculate_z_serial_purepython)
                                                             2    0.000    0.000  test_cpu_time.py:24(calc_pure_python)
test_cpu_time.py:24(calc_pure_python)               <-       1    1.246   21.948  test_cpu_time.py:1(<module>)
{abs}                                               <- 34219980    3.692    3.692  test_cpu_time.py:10(calculate_z_serial_purepython)
{sum}                  

<pstats.Stats instance at 0x115d2eb00>

In [8]:
p.print_callees()

   Random listing order was used

Function                                            called...
                                                        ncalls  tottime  cumtime
{method 'append' of 'list' objects}                 ->
{time.time}                                         ->
test_cpu_time.py:1(<module>)                        ->       1    1.246   21.948  test_cpu_time.py:24(calc_pure_python)
{len}                                               ->
test_cpu_time.py:24(calc_pure_python)               ->       1   16.739   20.503  test_cpu_time.py:10(calculate_z_serial_purepython)
                                                             2    0.000    0.000  {len}
                                                       2002000    0.192    0.192  {method 'append' of 'list' objects}
                                                             1    0.007    0.007  {sum}
                                                             2    0.000    0.000  {time.time}
{abs}            

<pstats.Stats instance at 0x115d2eb00>

## Using line_profiler for CPU profile line by line
- run this command to install line_profiler: *pip install line_profiler*
- run this command to profile cpu time: *kernprof -l -v test_cpu_time.py*


In [ ]:
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 144.420880079 seconds
Wrote profile results to test_cpu_time.py.lprof
Timer unit: 1e-06 s

Total time: 84.8698 s
File: test_cpu_time.py
Function: calculate_z_serial_purepython at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     9                                           @profile
    10                                           def calculate_z_serial_purepython(maxiter, zs, cs):
    11                                               """Calculate output list using Julia update rule"""
    12         1         9095   9095.0      0.0      output = [0] * len(zs)
    13   1000001       864289      0.9      1.0      for i in range(len(zs)):
    14   1000000       630460      0.6      0.7          n = 0
    15   1000000       826833      0.8      1.0          z = zs[i]
    16   1000000       825526      0.8      1.0          c = cs[i]
    17  34219980     31922079      0.9     37.6          while n < maxiter and abs(z) < 2:
    18  33219980     26003446      0.8     30.6              z = z * z + c
    19  33219980     22857267      0.7     26.9              n += 1
    20   1000000       930844      0.9      1.1          output[i] = n
    21         1            9      9.0      0.0      return output

## Using memory_profiler for RAM profiler
This run 10x - 100x slower than line_profiler, so install additional library to speed up. So use it sometimes
- run this command to install psutil: *pip install psutil*
- run this command to install memory_profiler: *pip install memory_profiler*
- run this command to profile memory: *python -m memory_profiler test_cpu_time.py*

In [ ]:
Filename: test_cpu_time.py

Line #    Mem usage    Increment   Line Contents
================================================
     9   89.531 MiB    0.000 MiB   @profile
    10                             def calculate_z_serial_purepython(maxiter, zs, cs):
    11                                 """Calculate output list using Julia update rule"""
    12   97.164 MiB    7.633 MiB       output = [0] * len(zs)
    13  145.633 MiB   48.469 MiB       for i in range(len(zs)):
    14  145.633 MiB    0.000 MiB           n = maxiter
    15  145.633 MiB    0.000 MiB           z = zs[i]
    16  145.633 MiB    0.000 MiB           c = cs[i]
    17  145.633 MiB    0.000 MiB           while n > 0 and abs(z) < 2:
    18  145.633 MiB    0.000 MiB               z = z * z + c
    19  145.633 MiB    0.000 MiB               n -= 1
    20  145.633 MiB    0.000 MiB           output[i] = n
    21  138.000 MiB   -7.633 MiB       return output
